In [1]:
def read_file(file):
    with open(file) as f:
        contents = f.readlines()

    attributes = contents[2: contents.index('@data\n')]
    data = contents[contents.index('@data\n') + 1:]

    return attributes, data


attributes, data = read_file('./V4 data/2020.arff')
print(attributes)

In [ ]:
def parse_data(attributes, data):
    pass